### Metric Calculations
First we need to load a connection against the proper ES instance. We use an external module to load credentials from a file that will not be shared. If you want to run this, please use your own credentials, just put them in a file named '.settings' (in the same directory as this notebook) following the example file 'settings.sample'.

To work with **survey results** this notebook expect to find the following files:
* **UUID's**: ../data/uuids.csv
* **SURVEY**: ../data/survey-fake.csv

**If you need to use a different survey file, just modify in the second code cell below this one.**

In [1]:
import pandas

import util as ut

from util import ESConnection

es_conn = ESConnection()

In [26]:
# Read survey dataframe from survey and (response id, uuid) correspondence files

############# MODIFY YOUR SURVEY CSV FILE HERE ######################################
survey_df = ut.load_survey_df(survey_filepath='../data/survey-fake.csv',
                              uuids_filepath='../data/uuids.csv')
#####################################################################################

print('Identities with UUID found in survey file: ', len(survey_df))
print('Unique identities found: ', len(survey_df.uuid.unique()))

def add_common_filters(source, s):
    s = s.filter('terms', author_uuid=survey_df['uuid'].tolist())
    # EXCLUDE MOZILLA EMPLOYEES
    s = s.exclude('terms', author_org_name=['Mozilla Staff', 'Code Sheriff', 'Unknown'])
    #print(len(survey_df['uuid'].unique()))

    # Retrieve emails before given year
    #if (source == 'bugzilla'):
    #    s = s.filter('range', creation_ts={'lt': 'now/y'})
    #else:
    #    s = s.filter('range', grimoire_creation_date={'lt': 'now/y'})
        
    return s

Identities with UUID found in survey file:  327
Unique identities found:  327


In [27]:
survey_df

,uuid,active,age,country,gender,disability,education level,language,english proficiency,coding
0,8e750cd848ad5a102edcadd5b3e93c928db87b06,"Yes, I have contributed within the past year.",18-29,United States,Another Gender (write in),I wish to voluntarily share that my disability...,Prefer not to say,Other (please specify),Strong,Coding
1,e911a3c6fb7e95aa13899a09dcbf32ae94095c3d,"Yes, I have contributed within the past year.",18-29,Zimbabwe,Male,I am not a person with a disability,Bachelor’s Degree,English,Extremely strong,
2,7c63ef173f359e3b75e42e10dcec4716eebab452,"Yes, I have contributed within the past year.",18-29,Venezuela,Prefer not to say,I am not a person with a disability,Bachelor’s Degree,Other (please specify),Strong,Coding
3,046a18bb30a059adf370af956ad52e41f96851b0,"Yes, I have contributed within the past year.",30-39,Venezuela,Male,I am not a person with a disability,Bachelor’s Degree,English,Neither strong nor weak,
4,10c7ab6a3b37f0a67bc19c572678a2f8731c621b,"No, I have not contributed within the past yea...",18-29,Uruguay,Male,,Bachelor’s Degree,Hindi,Strong,
5,18f5f7c104725182c244c42bd3621756703e0c9c,"Yes, I have contributed within the past year.",18-29,Uruguay,Male,I am not a person with a disability,Bachelor’s Degree,English,Strong,
6,c88512a42bbe7d9b1d283adbfd59a45499343583,"Yes, I have contributed within the past year.",30-39,United States,Female,I am not a person with a disability,Postgraduate Degree,English,Strong,
7,02dc242268e8632b3069471be829025be76fed8a,"Yes, I have contributed within the past year.",18-29,United States,Male,I am not a person with a disability,Bachelor’s Degree,Other (please specify),Neither strong nor weak,Coding
8,23f18b9d13fe1cc102c5dff690a6ca2eb2890ba3,"Yes, I have contributed within the past year.",18-29,United States,Female,,Bachelor’s Degree,Hindi,Strong,Coding
9,0cd3d032f400e26364e884e768187a99f8d84556,"No, I have not contributed within the past yea...",65+,United States,Male,I am not a person with a disability,Postgraduate Degree,English,Extremely strong,


In [28]:
### GET AUTHORS IN GIT, THEIR PROJECTS AND COMMITS
source = 'git'
s = ut.create_search(es_conn, source)

s = add_common_filters(source, s)

# Bucketize by uuid and project
s.aggs.bucket('uuid', 'terms', field='author_uuid', size=100000) \
        .bucket('project', 'terms', field='project', size=100000) \
        .metric('commits', 'cardinality', field='hash', precision_threshold=1000000)

result = s.execute()

author_commits_df = ut.stack_by(result=result, group_field='uuid', subgroup_field='project', value_field='commits',
                                group_column='uuid', subgroup_column='git project', value_column='commits')

print('AUTHORS FOUND IN GIT: ', len(author_commits_df['uuid'].unique()))

AUTHORS FOUND IN GIT:  153


In [29]:
### GET AUTHORS IN BUGZILLA, THEIR PROJECTS AND NUMBER OF BUGS
source = 'bugzilla'
s = ut.create_search(es_conn, source)

s = add_common_filters(source, s)

# Bucketize by uuid and project
s.aggs.bucket('uuid', 'terms', field='author_uuid', size=100000) \
        .bucket('project', 'terms', field='project', size=100000)

result = s.execute()

author_bugs_df = ut.stack_by(result=result, group_field='uuid', subgroup_field='project',
                                group_column='uuid', subgroup_column='bugzilla project', value_column='bugs')

print('AUTHORS FOUND IN BUGZILLA: ', len(author_bugs_df['uuid'].unique()))

AUTHORS FOUND IN BUGZILLA:  177


In [30]:
### GET AUTHORS IN MAILING LISTS, THEIR PROJECTS AND NUMBER OF EMAILS
source = 'mbox'
s = ut.create_search(es_conn, source)

s = add_common_filters(source, s)

# Bucketize by uuid and project
s.aggs.bucket('uuid', 'terms', field='author_uuid', size=100000) \
        .bucket('project', 'terms', field='project', size=100000)

result = s.execute()

author_emails_df = ut.stack_by(result=result, group_field='uuid', subgroup_field='project',
                                group_column='uuid', subgroup_column='mbox project', value_column='emails')

print('AUTHORS FOUND IN MBOX: ', len(author_emails_df['uuid'].unique()))

AUTHORS FOUND IN MBOX:  122


In [31]:
### GET AUTHORS IN DISCOURSE, THEIR PROJECTS AND NUMBER OF MESSAGES
source = 'discourse'
s = ut.create_search(es_conn, source)

s = add_common_filters(source,s)

# Bucketize by uuid and project
s.aggs.bucket('uuid', 'terms', field='author_uuid', size=100000) \
        .bucket('project', 'terms', field='project', size=100000)

result = s.execute()

author_discourse_df = ut.stack_by(result=result, group_field='uuid', subgroup_field='project',
                                group_column='uuid', subgroup_column='discourse project', value_column='messages')

print('AUTHORS FOUND IN DISCOURSE: ', len(author_discourse_df['uuid'].unique()))

AUTHORS FOUND IN DISCOURSE:  85


In [32]:
### GET AUTHORS IN GITHUB, THEIR PROJECTS AND NUMBER OF ISSUES AND PR's
source = 'github_issues'

# PULL REQUESTS
s_prs = ut.create_search(es_conn, source)

s_prs = add_common_filters(source, s_prs)

# Get only PRs
s_prs = s_prs.filter('terms', pull_request=['true'])

s_prs.aggs.bucket('uuid', 'terms', field='author_uuid', size=100000) \
    .bucket('project', 'terms', field='project', size=100)
result_prs = s_prs.execute()


# ISSUES
s_iss = ut.create_search(es_conn, 'github_issues')

s_iss = add_common_filters(source, s_iss)

# Get only Issues
s_iss = s_iss.filter('terms', pull_request=['false'])
s_iss.aggs.bucket('uuid', 'terms', field='author_uuid', size=100000) \
    .bucket('project', 'terms', field='project', size=100)
result_iss = s_iss.execute()


# MERGE PR's and ISSUES

prs_df = ut.stack_by(result=result_prs, group_column='uuid', subgroup_column='github project',
                     value_column='pull requests', group_field='uuid', subgroup_field='project')
iss_df = ut.stack_by(result=result_iss, group_column='uuid', subgroup_column='github project',
                     value_column='issues', group_field='uuid', subgroup_field='project')

github_df = pandas.merge(prs_df, iss_df, how='outer', on=['uuid', 'github project'])
github_df = github_df.fillna(0)

print('AUTHORS FOUND IN GITHUB: ', len(github_df['uuid'].unique()))

AUTHORS FOUND IN GITHUB:  165


In [33]:
authors_global_df = pandas.concat([author_commits_df, 
                                   author_emails_df, 
                                   github_df, 
                                   author_bugs_df,
                                   author_discourse_df], 
                                  keys=['git', 'mbox', 'github', 'bugzilla', 'discourse'])


grouped_df = authors_global_df.groupby(['uuid']).agg({'emails': 'sum', 
                                                     'commits': 'sum',
                                                     'pull requests': 'sum',
                                                     'issues': 'sum',
                                                     'bugs': 'sum',
                                                     'messages': 'sum',
                                                     'git project': pandas.Series.nunique,
                                                     'mbox project': pandas.Series.nunique,
                                                     'github project': pandas.Series.nunique,
                                                     'bugzilla project': pandas.Series.nunique,
                                                     'discourse project': pandas.Series.nunique
                                                     })

grouped_df = grouped_df.reset_index().sort_values(by=['commits'], ascending=[False])


grouped_df = grouped_df[['uuid', 'commits', 'pull requests', 'issues', 'bugs', 'emails', 'messages',
                         'git project', 'github project', 'bugzilla project', 'mbox project',
                         'discourse project']]

print('AUTHORS FOUND IN GIT, GITHUB, BUGZILLA, MBOX, DISCOURSE: ', len(grouped_df['uuid'].unique()))

grouped_df = grouped_df.fillna(0)
grouped_df

AUTHORS FOUND IN GIT, GITHUB, BUGZILLA, MBOX, DISCOURSE:  269


,uuid,commits,pull requests,issues,bugs,emails,messages,git project,github project,bugzilla project,mbox project,discourse project
224,9b981479e05cbba272b3d2376a90143e25c0de81,1498.0,29.0,207.0,21.0,223.0,77.0,4,2,8,7,4
201,78d455c94f2585166eede53d9beb5bca20b7ecdc,892.0,204.0,251.0,0.0,0.0,0.0,2,1,0,0,0
236,aa0f0ae660df1027074d82681e9afa1874ea80b3,631.0,22.0,15.0,235.0,13.0,0.0,11,4,13,3,0
219,93a342f7f47d72ebeb9534d06cae0da7008c4bcd,591.0,69.0,232.0,80.0,412.0,613.0,9,8,11,4,10
89,26bf4956a4120517eaf322d07665984687c24d31,574.0,0.0,5.0,17.0,20.0,0.0,6,4,7,2,0
6,0257e946520d0e889519cb1a757006efc6202ca9,469.0,1.0,0.0,2.0,0.0,8.0,7,1,1,0,1
132,464afc233868c436f1ae7ea2470d687067bab5b3,397.0,50.0,2.0,347.0,212.0,0.0,9,3,13,2,0
187,700ebc9e2e03a453bc0048ebb3f167de6c5195ee,349.0,16.0,65.0,49.0,5.0,139.0,7,5,11,1,3
34,10e40bc95a121a91b53513dddb255daf512955a8,312.0,0.0,0.0,3.0,15.0,0.0,7,0,2,1,0
237,ab7d4d184923798a042a6b29eea633c9d52d546b,261.0,0.0,0.0,3.0,1.0,0.0,2,0,2,1,0


In [35]:
### USERS WITH NO ACTIVITY OR COLLABORATING IN UNTRACKED PROJECTS
print('Authors matched in Survey: ', len(survey_df.uuid.unique()))
print('Authors found in indexes & projects: ', len(grouped_df.uuid.unique()))
print('\nAuthors matched in survey but being employees or not found in any tracked project:\n')
for element in survey_df.uuid.unique():
    if element not in grouped_df.uuid.unique():
        print(element)

Authors matched in Survey:  327
Authors found in indexes & projects:  269

Authors matched in survey but being employees or not found in any tracked project:

531daed0a68236d39d6790ed7b80d3cdfeb7dd40
016dd3752addfe79920fcaa2934d0961c510d116
5a0d336e645d27220fdf47426e0618776db8f5d7
24b889d8fe3052177c78f8ddb5bcc7e95a5b55e7
829fa5bd85db080fed69156279859eaa2b43f28c
3a78b16f04acf84b7ba14cb07e1040a23dd47bdb
0183bd06d487bf71fd39a568ae892e3ec3079d82
3920385860048b31cbf8fa86fb94253d81ce7b11
80cc7c48f96c5f88398d194b719cf52be40e4964
076e15e27bc2a7eef3fdcd753a8cc1f2cdcf644c
6ba47dca3af4b299f9c37ca3ca47a6cc50da1eba
3623f40be1a6516b1e8b4d44fe22b3bfbbf19252
2549772b2c50f0947af7278ae87eb7de07efd6ad
5a6149af27d2e9ee748ad81a048c6367e833fc87
a2bf2639f553fe99c6ce52fcf53d4a71cd03cc30
5bee3c3d6b83251342adf69ad3af2db1ddd122b4
cbf91b5771b5d8dec38f1c67d7fe3a0fe5ec6e36
0dd6ba066630a92f7d4e12ec6c89254a7f2af498
73aee2ccaf24929a191f025edf58bf6e604bf64a
5eb30f7c3734ff5843467f318d6181113cb6c89a
dcfa4e4ec7f5941493e5e